In [ ]:
!pip install langchain-community # install the package

In [ ]:
!pip install pinecone-client # install the package

In [ ]:
!pip install langchain-pinecone # install the package

In [ ]:
!pip install langchain-cohere # install the package


In [ ]:
!pip install pypdf # install the package

In [ ]:



from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

from langchain.chains import RetrievalQA
from langchain_cohere import ChatCohere, CohereEmbeddings



os.environ["COHERE_API_KEY"] = " YOUR API KEY "
os.environ['PINECONE_API_KEY'] = " YOUR API KEY "


# Set your Pinecone API key
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Create or connect to the Pinecone index
index_name = "docs-rag-chatbot" # changed to lowercase and replaced spaces with hyphens
pc = Pinecone(api_key=pinecone_api_key)
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust dimensionality as needed
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)



loader = PyPDFLoader("/content/budget_speech.pdf")  # Adjust the path to your PDF file
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


embeddings = CohereEmbeddings(model="embed-english-light-v3.0")

vectorstore_from_docs = PineconeVectorStore.from_documents(
        docs,
        index_name=index_name,
        embedding=embeddings
    )


llm = ChatCohere()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_from_docs.as_retriever()
)



result = qa.invoke(input("Enter Query:"))
print("Answer:",result['result'])





Enter Query:what is Energy Security?
Answer: Energy security refers to the uninterrupted availability of energy sources at an affordable price. It is a crucial aspect of a country's economic and political stability, as it ensures that the nation has access to the energy resources it needs to function and grow. 

In the context of the provided text, Energy Security is one of the priorities outlined in the budget announcement. The strategy aims to achieve high and resource-efficient economic growth while ensuring energy security in terms of availability, accessibility, and affordability. This involves a comprehensive approach to energy transition, including the development and implementation of various energy sources and technologies.
